In [143]:
import pandas as pd
import json
import ast
import numpy as np
import datetime

### 1. Conectarse al enlace

In [127]:
#Read the data as json file from the url given
df = pd.read_json('https://api.stackexchange.com/2.2/search?order=desc&sort=activity&intitle=perl&site=stackoverflow')

In [136]:
df.head() 

,items,has_more,quota_max,quota_remaining
0,"{'tags': ['zsh'], 'owner': {'reputation': 164,...",True,300,272
1,"{'tags': ['windows', 'perl'], 'owner': {'reput...",True,300,272
2,"{'tags': ['regex', 'google-bigquery'], 'owner'...",True,300,272
3,"{'tags': ['perl', 'cgi'], 'owner': {'reputatio...",True,300,272
4,"{'tags': ['perl'], 'owner': {'reputation': 83,...",True,300,272


As shown above, we can observer that the actually data is nested in the column "items" of our pandas dataframe.
We need to extract this data as is the information that we actually need.

In [131]:
#In a new dataframe I gonna extract the json files from the "items" columns with the method json_normalize
data = pd.DataFrame()
data = pd.io.json.json_normalize(df['items'])

C:\Users\Gerardo\AppData\Local\Temp/ipykernel_155968/211277782.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(df['items'])


In [137]:
#The dataframe extracted 
data.head()

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
0,[zsh],True,44,3,1,1660254064,1660065551,1.660076e+09,73295790,CC BY-SA 4.0,...,164.0,2194258.0,registered,https://i.stack.imgur.com/QHQsu.png?s=256&g=1,GBelanger,https://stackoverflow.com/users/2194258/gbelanger,NaN,NaN,NaN,NaN
1,"[windows, perl]",True,70,1,2,1660134590,1659970034,1.660046e+09,73279792,CC BY-SA 4.0,...,498.0,11365539.0,registered,https://lh6.googleusercontent.com/-B5YB6Qt0o1U...,Warpstar22,https://stackoverflow.com/users/11365539/warps...,73306362.0,NaN,NaN,NaN
2,"[regex, google-bigquery]",True,30,1,0,1660131016,1660128366,1.660129e+09,73304896,CC BY-SA 4.0,...,11.0,18598417.0,registered,https://www.gravatar.com/avatar/0f516e637ff295...,Amethyst,https://stackoverflow.com/users/18598417/amethyst,73305007.0,NaN,NaN,NaN
3,"[perl, cgi]",True,40,1,-2,1660063216,1659711389,1.659712e+09,73251689,CC BY-SA 4.0,...,235.0,10704132.0,registered,https://www.gravatar.com/avatar/b78ac0af40e7ae...,santa100,https://stackoverflow.com/users/10704132/santa100,NaN,NaN,NaN,NaN
4,[perl],False,74,0,0,1660058528,1659956676,1.660059e+09,73276877,NaN,...,83.0,18372990.0,registered,https://www.gravatar.com/avatar/af15e2de571edb...,ibrahim muhmud usmani,https://stackoverflow.com/users/18372990/ibrah...,NaN,1.659977e+09,Needs details or clarity,NaN


In [159]:
data.columns

Index(['tags', 'is_answered', 'view_count', 'answer_count', 'score',
       'last_activity_date', 'creation_date', 'last_edit_date', 'question_id',
       'content_license', 'link', 'title', 'owner.reputation', 'owner.user_id',
       'owner.user_type', 'owner.profile_image', 'owner.display_name',
       'owner.link', 'accepted_answer_id', 'closed_date', 'closed_reason',
       'owner.accept_rate'],
      dtype='object')

### 2. Obtener el numero de respuestas contestadas y no contestadas

In [135]:
#We count the values from the "is_answered" columns 
#Which arrows us that 26 posts were answered while 4 of them were ignored them
data.is_answered.value_counts()

True     26
False     4
Name: is_answered, dtype: int64

### 3. Obtener la respuesta con menor numero de vistas

In [167]:
less_view = data["view_count"].min()

data.loc[data.view_count == less_view ]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
8,"[perl, conda, slurm]",False,26,0,1,1660045700,1970-01-01 00:00:01.659957476,1.660046e+09,73277012,NaN,...,23.0,17665505.0,registered,https://www.gravatar.com/avatar/a7702facff15bc...,btredcup,https://stackoverflow.com/users/17665505/btredcup,NaN,1.660046e+09,Duplicate,NaN


### 4. Obtener la respuesta mas vieja y mas actual

In [147]:
data['creation_date'] = pd.to_datetime(data['creation_date'], format = "%Y%m%d")

In [149]:
data['creation_date'].max(), data['creation_date'].min()

(Timestamp('1970-01-01 00:00:01.660128366'),
 Timestamp('1970-01-01 00:00:01.283535766'))

### 5. Obtener la respuesta del owner que tenga una mayor reputación

In [168]:
best_reputation = data["owner.reputation"].max()
data.loc[data['owner.reputation'] == best_reputation]


,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
9,"[raku, rakudo, rakudo-star, zef]",True,881,2,3,1660031009,1970-01-01 00:00:01.514920426,1.529825e+09,48066536,CC BY-SA 4.0,...,5540.0,7730888.0,registered,https://www.gravatar.com/avatar/2006b814065584...,Eugene Barsky,https://stackoverflow.com/users/7730888/eugene...,48078157.0,NaN,NaN,85.0


### 6. Imprimir en consola del punto 2 al 5

In [155]:
data.iloc[2:6]

,tags,is_answered,view_count,answer_count,score,last_activity_date,creation_date,last_edit_date,question_id,content_license,...,owner.reputation,owner.user_id,owner.user_type,owner.profile_image,owner.display_name,owner.link,accepted_answer_id,closed_date,closed_reason,owner.accept_rate
2,"[regex, google-bigquery]",True,30,1,0,1660131016,1970-01-01 00:00:01.660128366,1.660129e+09,73304896,CC BY-SA 4.0,...,11.0,18598417.0,registered,https://www.gravatar.com/avatar/0f516e637ff295...,Amethyst,https://stackoverflow.com/users/18598417/amethyst,73305007.0,NaN,NaN,NaN
3,"[perl, cgi]",True,40,1,-2,1660063216,1970-01-01 00:00:01.659711389,1.659712e+09,73251689,CC BY-SA 4.0,...,235.0,10704132.0,registered,https://www.gravatar.com/avatar/b78ac0af40e7ae...,santa100,https://stackoverflow.com/users/10704132/santa100,NaN,NaN,NaN,NaN
4,[perl],False,74,0,0,1660058528,1970-01-01 00:00:01.659956676,1.660059e+09,73276877,NaN,...,83.0,18372990.0,registered,https://www.gravatar.com/avatar/af15e2de571edb...,ibrahim muhmud usmani,https://stackoverflow.com/users/18372990/ibrah...,NaN,1.659977e+09,Needs details or clarity,NaN
5,[perl],False,70,0,-1,1660058318,1970-01-01 00:00:01.660038872,1.660058e+09,73289843,CC BY-SA 4.0,...,83.0,18372990.0,registered,https://www.gravatar.com/avatar/af15e2de571edb...,ibrahim muhmud usmani,https://stackoverflow.com/users/18372990/ibrah...,NaN,NaN,NaN,NaN
